# Home Sale Forecasting
In this tutorial, we demonstrate how to use the forecasting capablity of EvaDB to predict the home sale price.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/16-homesale-forecasting.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/16-homesale-forecasting.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/16-homesale-forecasting.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

## Setup
We first setup the backend postgres database and EvaDB to bring AI inside database systems.

### Start Postgres

In [2]:
!apt -qq install postgresql
!service postgresql start

postgresql is already the newest version (14+238).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
 * Starting PostgreSQL 14 database server
   ...done.


### Create User and Database

In [3]:
!sudo -u postgres psql -c "CREATE USER eva WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

CREATE ROLE
CREATE DATABASE


###Prettify  Output

In [4]:
import warnings
warnings.filterwarnings("ignore")

### Install EvaDB

We install EvaDB with extra postgres and forecasting dependency.

In [5]:
%pip install --quiet "evadb[postgres,forecasting] @ git+https://github.com/georgia-tech-db/evadb.git@staging"

import evadb
cursor = evadb.connect().cursor()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.0/275.0 kB

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 1.04MB/s]
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


## Prepare Data
We then prepara the dataset used in this time serise forecasting use case.

### Create Data Source in EvaDB
We use data source to connect EvaDB directly to underlying database systems like Postgres.

In [7]:
params = {
    "user": "eva",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

10-12-2023 06:11:59 ERROR [plan_executor:plan_executor.py:execute_plan:0179] postgres_data already exists.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/plan_executor.py", line 175, in execute_plan
    yield from output
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/create_database_executor.py", line 42, in exec
    raise ExecutorError(f"{self.node.database_name} already exists.")
evadb.executor.executor_utils.ExecutorError: postgres_data already exists.
ERROR:evadb.utils.logging_manager:postgres_data already exists.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/plan_executor.py", line 175, in execute_plan
    yield from output
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/create_database_executor.py", line 42, in exec
    raise ExecutorError(f"{self.node.database_name} already exists.")
evadb.executor.executor_utils.ExecutorError: postgres_data alrea

ExecutorError: ignored

### Load the Datasets
We load the [House Property Sales Time Series](https://www.kaggle.com/datasets/htagholdings/property-sales?resource=download) into our PostgreSQL database.

In [8]:
!mkdir -p content
!wget -qnc -O /content/home_sales.csv https://www.dropbox.com/scl/fi/2e9yyzymm0rwzria2kvzo/raw_sales.csv?rlkey=lfdr9th7csw7ru42mtaw00hx1&dl=0

In [9]:
cursor.query("""
  USE postgres_data {
    CREATE TABLE IF NOT EXISTS home_sales (datesold VARCHAR(64), postcode INT, price INT, propertyType VARCHAR(64), bedrooms INT)
  }
""").df()

,status
0,success


In [10]:
cursor.query("""
  USE postgres_data {
    COPY home_sales(datesold, postcode, price, propertyType, bedrooms)
    FROM '/content/home_sales.csv'
    DELIMITER ',' CSV HEADER
  }
""").df()

,status
0,success


### Preview the Data
The `home_sales` table contains 4 columns.
- postcode: 4 digit postcode of the suburb where the property was sold
- price: Price for which the property was sold
- bedrooms: Number of bedrooms
- datesold: Date on which this property was sold
- propertytype: Property type i.e. house or unit

In [11]:
cursor.query("SELECT * FROM postgres_data.home_sales LIMIT 3;").df()

,postcode,price,bedrooms,datesold,propertytype
0,2607,525000,4,2007-02-07 00:00:00,house
1,2906,290000,3,2007-02-27 00:00:00,house
2,2905,328000,3,2007-03-07 00:00:00,house


## Analysis Data with EvaDB

We then use EvaDB to train a model to forecast the home price.

### Train the Forecast Model
We use the [statsforecast](https://github.com/Nixtla/statsforecast) engine to train a time serise forecast model for sale prices of home with two bedrooms.

In [12]:
cursor.query("""
  CREATE OR REPLACE FUNCTION HomeSaleForecast FROM
    (
      SELECT propertytype, datesold, price
      FROM postgres_data.home_sales
      WHERE bedrooms = 3 AND postcode = 2607
    )
  TYPE Forecasting
  PREDICT 'price'
  HORIZON 3
  TIME 'datesold'
  ID 'propertytype'
  FREQUENCY 'W'
""").df()

,0
0,Function HomeSaleForecast added to the database.


### Use the Forecast Model
We then use the `HomeSaleForecast` model to predict the sale price for homes with two bedrooms for the next three month.

In [13]:
cursor.query("SELECT HomeSaleForecast();").df()

,propertytype,datesold,price
0,house,2019-07-21,766572.93750
1,house,2019-07-28,766572.93750
2,house,2019-08-04,766572.93750
3,unit,2018-12-23,417229.78125
4,unit,2018-12-30,409601.65625
5,unit,2019-01-06,402112.96875


We can use `ORDER BY` to find out the type of home and months that have lower market price.

In [14]:
cursor.query("SELECT HomeSaleForecast() ORDER BY price;").df()

,propertytype,datesold,price
0,unit,2019-01-06,402112.96875
1,unit,2018-12-30,409601.65625
2,unit,2018-12-23,417229.78125
3,house,2019-07-21,766572.93750
4,house,2019-07-28,766572.93750
5,house,2019-08-04,766572.93750


### Try Neuralforecast instead
By default, EvaDB uses [statsforecast](https://github.com/Nixtla/statsforecast) for time series forecasting. We notice there are flat predictions for the `propertytype = house`. Let's try [Neuralforecast](https://github.com/Nixtla/neuralforecast) instead.

### Tune the Forecast Frequency
If we do not provide frequency, EvaDB will try to infer the frequency.

In [15]:
cursor.query("""
  CREATE OR REPLACE FUNCTION HomeSaleForecast FROM
    (
      SELECT propertytype, datesold, price
      FROM postgres_data.home_sales
      WHERE bedrooms = 3 AND postcode = 2607
    )
  TYPE Forecasting
  PREDICT 'price'
  HORIZON 3
  TIME 'datesold'
  ID 'propertytype'
""").df()

10-12-2023 06:21:49 ERROR [plan_executor:plan_executor.py:execute_plan:0179] Can not infer the frequency for HomeSaleForecast. Please explicitly set it.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/plan_executor.py", line 175, in execute_plan
    yield from output
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/create_function_executor.py", line 526, in exec
    ) = self.handle_forecasting_function()
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/create_function_executor.py", line 254, in handle_forecasting_function
    raise RuntimeError(
RuntimeError: Can not infer the frequency for HomeSaleForecast. Please explicitly set it.
ERROR:evadb.utils.logging_manager:Can not infer the frequency for HomeSaleForecast. Please explicitly set it.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/plan_executor.py", line 175, in execute_plan
    yield from output
  Fil

ExecutorError: ignored

In this case, we can not infer the frequency from the dataset. Let's try manully setting the frequency to `M` (i.e., month end frequency).

In [16]:
cursor.query("""
  CREATE OR REPLACE FUNCTION HomeSaleForecast FROM
    (
      SELECT propertytype, datesold, price
      FROM postgres_data.home_sales
      WHERE bedrooms = 3 AND postcode = 2607
    )
  TYPE Forecasting
  PREDICT 'price'
  HORIZON 3
  TIME 'datesold'
  ID 'propertytype'
  FREQUENCY 'M'
""").df()

,0
0,Function HomeSaleForecast added to the database.


After we change the `Frequency` to `M`, the horizon or the forecasting steps have also become months. We also notice that there are no flat predictions.  

In [17]:
cursor.query("SELECT HomeSaleForecast() ORDER BY price;").df()

,propertytype,datesold,price
0,unit,2019-02-28,402112.96875
1,unit,2019-01-31,409601.65625
2,unit,2018-12-31,417229.78125
3,house,2019-08-31,758721.31250
4,house,2019-07-31,765442.31250
5,house,2019-09-30,771323.18750
